In [64]:

import numpy as np
import pandas as pd
url='https://raw.githubusercontent.com/ACM-NITK/Machine-Learning-SMP-2019/master/Assignment%201/PL18-19.csv'
df = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe
df.columns

Index([u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR', u'HTHG',
       u'HTAG', u'HTR', u'Referee', u'HS', u'AS', u'HST', u'AST', u'HF', u'AF',
       u'HC', u'AC', u'HY', u'AY', u'HR', u'AR'],
      dtype='object')

In [0]:
df=df.rename(columns={'FTHG':'HomeTeamGoals','FTAG':'AwayTeamGoals','FTR':'FullTimeResult','HTHG':'HalfTimeHTGoals','HTAG':'HalfTimeATGoals','HTR':'HalfTimeResult','HS':'HTShots','AS':'ATShots','HST':'HTShotsOnTarget','AST':'ATShotsOnTarget','HF':'HomeTeamGoalsFouls','AF':'AwayTeamFouls','HC':'HomeTeamCorners','AC':'AwayTeamCorners','HY':'HTYellowCards','AY':'ATYellowCards','HR':'HTRedCards','AR':'ATRedCards'})
#Renaming Columns

In [66]:
print(np.union1d(df['HomeTeam'], df['AwayTeam']))

['Arsenal' 'Bournemouth' 'Brighton' 'Burnley' 'Cardiff' 'Chelsea'
 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield' 'Leicester'
 'Liverpool' 'Man City' 'Man United' 'Newcastle' 'Southampton' 'Tottenham'
 'Watford' 'West Ham' 'Wolves']


In [67]:
df['TotalShots']=df['HTShots']+df['ATShots']
df['TotalShots'].max()


42

In [68]:
df['TotalShotsOnTarget']=df['HTShotsOnTarget']+df['ATShotsOnTarget']
df['Ratio']=df['TotalShotsOnTarget']/df['TotalShots']
df.head()


,Date,HomeTeam,AwayTeam,HomeTeamGoals,AwayTeamGoals,FullTimeResult,HalfTimeHTGoals,HalfTimeATGoals,HalfTimeResult,Referee,HTShots,ATShots,HTShotsOnTarget,ATShotsOnTarget,HomeTeamGoalsFouls,AwayTeamFouls,HomeTeamCorners,AwayTeamCorners,HTYellowCards,ATYellowCards,HTRedCards,ATRedCards,TotalShots,TotalShotsOnTarget,Ratio
0,2018-08-10,Man United,Leicester,2,1,H,1,0,H,A Marriner,8,13,6,4,11,8,2,5,2,1,0,0,21,10,0.476190
1,2018-08-11,Bournemouth,Cardiff,2,0,H,1,0,H,K Friend,12,10,4,1,11,9,7,4,1,1,0,0,22,5,0.227273
2,2018-08-11,Fulham,Crystal Palace,0,2,A,0,1,A,M Dean,15,10,6,9,9,11,5,5,1,2,0,0,25,15,0.600000
3,2018-08-11,Huddersfield,Chelsea,0,3,A,0,2,A,C Kavanagh,6,13,1,4,9,8,2,5,2,1,0,0,19,5,0.263158
4,2018-08-11,Newcastle,Tottenham,1,2,A,1,2,A,M Atkinson,15,15,2,5,11,12,3,5,2,2,0,0,30,7,0.233333


In [69]:
df['Totalcards']=df['HTYellowCards']+df['ATYellowCards']+df['HTRedCards']+df['ATRedCards']
df.iloc[:,[9,25]].groupby(['Referee']).sum()


,Totalcards
Referee,
A Madley,2
A Marriner,76
A Taylor,103
C Kavanagh,76
C Pawson,99
D Coote,35
G Scott,39
J Linington,4
J Moss,98


In [70]:
df['HTCards']=df['HTYellowCards']+df['HTRedCards']
df['ATCards']=df['ATYellowCards']+df['ATRedCards']
a=df.iloc[:,[1,26]]
b=df.iloc[:,[2,27]]
a.columns=['Team','Cards']
b.columns=['Team','Cards']
x=pd.concat([a,b])
x.groupby(['Team']).sum()

,Cards
Team,
Arsenal,74
Bournemouth,61
Brighton,64
Burnley,76
Cardiff,66
Chelsea,50
Crystal Palace,60
Everton,59
Fulham,70


In [71]:
d={'H':1,'A':0,'D':0}
df['HomeWin']=df['FullTimeResult'].map(d)
a=df.iloc[:,[9,28]].groupby(['Referee'],as_index=False).sum()
a.loc[df['HomeWin'].idxmin()]

Referee    A Taylor
HomeWin          20
Name: 2, dtype: object

In [72]:
df['SameResult']=((df['FullTimeResult']!='D') & (df['HalfTimeResult']==df['FullTimeResult']) )
a=float((df['SameResult']==True).sum())
b=float((df['SameResult']==False).sum())
print((a*100)/(a+b))

48.9473684211


In [82]:
l={'H':3,'D':1,'A':0}
m={'H':0,'D':1,'A':3}
df['HomeScore']=df['FullTimeResult'].map(l)
df['AwayScore']=df['FullTimeResult'].map(m)
l={'H':1,'D':0,'A':0}
df['HomeWins']=df['FullTimeResult'].map(l)
l={'H':0,'D':0,'A':1}
df['AwayWins']=df['FullTimeResult'].map(l)
l={'H':0,'D':1,'A':0}
df['Draws']=df['FullTimeResult'].map(l)
a=df.groupby('HomeTeam',as_index=False).sum()
b=df.groupby('AwayTeam',as_index=False).sum()
x=a.loc[:,['HomeTeam','HomeScore','HomeWins','Draws','AwayWins','HomeTeamGoals','AwayTeamGoals']]
y=b.loc[:,['AwayTeam','AwayScore','AwayWins','Draws','HomeWins','AwayTeamGoals','HomeTeamGoals']]
x.columns=['Team','Score','Wins','Draws','Losses','GoalsFor','GoalsAgainst']
y.columns=['Team','Score','Wins','Draws','Losses','GoalsFor','GoalsAgainst']
final=pd.merge(x,y,on='Team')
final['Points']=final['Score_x']+final['Score_y']
final['Draws']=final['Draws_x']+final['Draws_y']
final['Wins']=final['Wins_x']+final['Wins_y']
final['Losses']=final['Losses_x']+final['Losses_y']
final['GoalsFor']=final['GoalsFor_x']+final['GoalsFor_y']
final['GoalsAgainst']=final['GoalsAgainst_x']+final['GoalsAgainst_y']
final['Played']=final['Losses']+final['Wins']+final['Draws']
final['GoalsDifference']=final['GoalsFor']-final['GoalsAgainst']
table=final.loc[:,['Team','Played','Wins','Losses','Draws','GoalsFor','GoalsAgainst','GoalsDifference','Points']]
table.sort_values(by='Points',ascending=False)



,Team,Played,Wins,Losses,Draws,GoalsFor,GoalsAgainst,GoalsDifference,Points
12,Man City,38,32,4,2,95,23,72,98
11,Liverpool,38,30,1,7,89,22,67,97
5,Chelsea,38,21,8,9,63,39,24,72
16,Tottenham,38,23,13,2,67,39,28,71
0,Arsenal,38,21,10,7,73,51,22,70
13,Man United,38,19,10,9,65,54,11,66
19,Wolves,38,16,13,9,47,46,1,57
7,Everton,38,15,14,9,54,46,8,54
18,West Ham,38,15,16,7,52,55,-3,52
10,Leicester,38,15,16,7,51,48,3,52
